<a href="https://colab.research.google.com/github/AlexeyTri/MyMachineLearningSeminars/blob/main/seminar21/%D0%9E%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D0%B5_P%D0%B0%D0%BD%D0%B6%D0%B8%D1%80%D0%BE%D0%B2%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip install --quiet -U  pyserini jsonlines scikit-surprise fastrank

In [ ]:
!pip install  faiss

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# DATA_PATH = './new_data'
# !mkdir new_data
# !cp /content/drive/MyDrive/'МФТИ машинное обучение'/'Домашки'/'2-й семестр'/'dataset for seminar 19'/lenta-ru-filtred.csv new_data
# !cp /content/drive/MyDrive/'МФТИ машинное обучение'/'Домашки'/'2-й семестр'/'dataset for seminar 19'/MSLR-WEB10K.zip new_data/
!unzip new_data/MSLR-WEB10K.zip -d new_data/mslr

Библиотеки

In [24]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import scipy
import json
import re
import os
import jsonlines
import lightgbm as lgb

from surprise import Dataset, Reader, SVD
from fastrank import CDataset, TrainRequest
from sklearn.datasets import load_svmlight_file
# строка кода не работает
# from pyserini.search import SimpleSearcher

# Learning to Rank

Загружаем DataSet

In [29]:
DATA_PATH = 'new_data/mslr/Fold1'
(X_train, y_train, qid_train) = load_svmlight_file(
    os.path.join(DATA_PATH, 'train.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_val, y_val, qid_val) = load_svmlight_file(
    os.path.join(DATA_PATH, 'vali.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_test, y_test, qid_test) = load_svmlight_file(
    os.path.join(DATA_PATH, 'test.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)

train_size = 50000
X_train = X_train.toarray()[:train_size]
y_train = y_train[:train_size]
qid_train = qid_train[:train_size]

val_size = 10000
X_val = X_val.toarray()[:val_size]
y_val = y_val[:val_size]
qid_val = qid_val[:val_size]

test_size = 1000
X_test = X_test.toarray()[:test_size]
y_test = y_test[:test_size]
qid_test = qid_test[:test_size]

In [30]:
ids = np.random.RandomState(42).randint(0, len(X_train), size=5)
(X_train[ids][:,:6], y_train[ids], qid_train[ids])

(array([[2.      , 0.      , 1.      , 1.      , 2.      , 1.      ],
        [3.      , 0.      , 3.      , 2.      , 3.      , 1.      ],
        [2.      , 1.      , 2.      , 0.      , 2.      , 1.      ],
        [2.      , 0.      , 1.      , 0.      , 2.      , 1.      ],
        [6.      , 2.      , 1.      , 1.      , 6.      , 0.857143]],
       dtype=float32),
 array([0., 0., 2., 0., 1.]),
 array([2071,  151, 4966, 5971, 1486]))

Определяем модель

In [31]:
train_request = TrainRequest.random_forest()
params = train_request.params
params.num_trees = 100
params.feature_sampling_rate = 0.5
params.instance_sampling_rate = 0.5

params.quit = True
params.seed =42

Специальный формат данных

In [32]:
dataset_train = CDataset.from_numpy(X_train, y_train, qid_train)
dataset_test = CDataset.from_numpy(X_test, y_test, qid_test)

features = dataset_train.feature_names()
train_queries = sorted(dataset_train.queries())
test_queries = sorted(dataset_test.queries())

train = dataset_train.subsample_queries(train_queries)
test = dataset_test.subsample_queries(test_queries)

In [33]:
model = train.train_model(train_request)